In [5]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import urllib.request
import pickle
import os
import json
import torch
import shutil
import Levenshtein
import requests

In [6]:
!pip install transformers
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#creating images (OLD FUNCTION, not used)
def img(text, path):
    img = Image.new('RGB', (150, 12))
    #img = Image.new('L', (150, 12))
    fnt = ImageFont.truetype(font, 10)
    d = ImageDraw.Draw(img)
    d.text((0, 0), text, font=fnt, fill = (255))
    enhancer = ImageEnhance.Contrast(img)
    im_output = enhancer.enhance(1.5)
    #transposed  = img.transpose(Image.ROTATE_90)
    im_output.save(path + text + '.jpg', 'JPEG')

In [8]:
font_path = 'font/NotoSansMono-Light.ttf'
def generate_image_with_text(text, path):
    # Imposta il font e il colore del testo
    #font = ImageFont.load_default()
    
    font = ImageFont.truetype(font_path, size=24)  # Imposta la dimensione del font desiderata
    color = (0, 0, 0)  # Nero
    
    # Crea una nuova immagine con uno sfondo bianco
    img = Image.new('RGB', (300, 110), color=(255, 255, 255))
    draw = ImageDraw.Draw(img)

    # Scrive il testo sull'immagine
    draw.text((10, 10), text, fill=color, font=font)

    # Salva l'immagine nel percorso specificato
    img.save(path)

In [9]:
#ratio
#70 : 20 : 10
#train : test : validation

In [10]:
os.mkdir('real_pics')
os.mkdir('real_pics/train')
os.mkdir('real_pics/test')
os.mkdir('real_pics/valid')
os.mkdir('fake_pics')
os.mkdir('fake_pics/train')
os.mkdir('fake_pics/test')
os.mkdir('fake_pics/valid')

FileExistsError: [Errno 17] File exists: 'real_pics'

In [ ]:
shutil.rmtree('real_pics/valid')
shutil.rmtree('real_pics/train')
shutil.rmtree('real_pics/test')
shutil.rmtree('real_pics')
shutil.rmtree('fake_pics/valid')
shutil.rmtree('fake_pics/train')
shutil.rmtree('fake_pics/test')
shutil.rmtree('fake_pics')


In [11]:
#real pics generation


#validation images
f = open('real.json')
data = json.load(f)
test_array = data["valid"]
for item in test_array:
  generate_image_with_text(item, f'real_pics/valid/{item}.jpg')
f.close()

#training images
f = open('real.json')
data = json.load(f)
test_array = data["train"]
for item in test_array:
  generate_image_with_text(item, f'real_pics/train/{item}.jpg')
f.close()

#testing images
f = open('real.json')
data = json.load(f)
test_array = data["test"]
for item in test_array:
  generate_image_with_text(item, f'real_pics/test/{item}.jpg')
f.close()

In [12]:
#fake pics generation

f = open('fake.json')
data = json.load(f)
test_array = data["valid"]
for item in test_array:
  generate_image_with_text(item, f'fake_pics/valid/{item}.jpg')
f.close()


f = open('fake.json')
data = json.load(f)
test_array = data["test"]
for item in test_array:
  generate_image_with_text(item, f'fake_pics/test/{item}.jpg')
f.close()


f = open('fake.json')
data = json.load(f)
test_array = data["train"]
for item in test_array:
  generate_image_with_text(item, f'fake_pics/train/{item}.jpg')
f.close()


In [13]:
def calculate_word_accuracy(ground_truth, prediction):
    ground_truth = ground_truth.replace(" ", "").lower()
    prediction = prediction.replace(" ", "").lower()

    edit_distance = Levenshtein.distance(ground_truth, prediction)

    return edit_distance  # Restituisce la distanza di Levenshtein

In [14]:
from transformers import (
    TrOCRConfig,
    TrOCRProcessor,
    TrOCRForCausalLM,
    ViTConfig,
    ViTModel,
    VisionEncoderDecoderModel,
)
# Creazione del modello ViT con configurazione predefinita
encoder = ViTModel(ViTConfig())

# Cartella contenente le immagini
#folder_path = 'real_pics/train'
folder_path_real_pics = 'real_pics/test'
folder_path_fake_pics = 'fake_pics/test'

# Creazione del decoder TrOCR
decoder = TrOCRForCausalLM(TrOCRConfig())

# Creazione del modello completo VisionEncoderDecoderModel
model = VisionEncoderDecoderModel(encoder=encoder, decoder=decoder)

# Caricamento del processor preaddestrato
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# Iterazione attraverso le immagini nella cartella
print('real pics training')
current_accuracy = 0
average_accuracy = 0
word_count = 0

word_accuracies = []
average_accuracies = []

for item in os.listdir(folder_path_real_pics):
    if item.endswith('.jpg'):
        image_path = os.path.join(folder_path_real_pics, item)
        image = Image.open(image_path)

        pixel_values = processor(image, return_tensors="pt").pixel_values

        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        generated_text = generated_text.replace(" ", "")
        item = item[:-4]

        edit_distance = calculate_word_accuracy(item, generated_text)

        # Calcola la precisione
        word_accuracy = 100 * (1 - (edit_distance / max(len(item), len(generated_text))))
        
        # Calcola la precisione media
        word_count += 1
        average_accuracy = ((word_count - 1) * average_accuracy + word_accuracy) / word_count

        word_accuracies.append(word_accuracy)
        average_accuracies.append(average_accuracy)

        print(f"Text generated from image {item}:")
        print(generated_text)
        print("The edit distance between '{}' and '{}' is: {}".format(item, generated_text, edit_distance))
        #print(f"Word Accuracy: {word_accuracy:.2f}%")
        #print(f"Average Accuracy: {average_accuracy:.2f}%")
        #print('\n')


print("############################")
"""
print('fake pics training')
for item in os.listdir(folder_path_fake_pics):
    if item.endswith('.jpg'):
        image_path = os.path.join(folder_path_fake_pics, item)
        image = Image.open(image_path)

        pixel_values = processor(image, return_tensors="pt").pixel_values

        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        generated_text = generated_text.replace(" ", "")
        item = item[:-4]

        edit_distance = calculate_word_accuracy(item, generated_text)

        # Calcola la precisione
        word_accuracy = 100 * (1 - (edit_distance / max(len(item), len(generated_text))))
        
        # Calcola la precisione media
        word_count += 1
        average_accuracy = ((word_count - 1) * average_accuracy + word_accuracy) / word_count

        word_accuracies.append(word_accuracy)
        average_accuracies.append(average_accuracy)

        print(f"Text generated from image {item}:")
        print(generated_text)
        print("The edit distance between '{}' and '{}' is: {}".format(item, generated_text, edit_distance))
        print(f"Word Accuracy: {word_accuracy:.2f}%")
        print(f"Average Accuracy: {average_accuracy:.2f}%")
        print('\n')

# Creazione del grafico
plt.plot(word_accuracies, label='Word Accuracy')
plt.plot(average_accuracies, label='Average Accuracy')
plt.xlabel('Image Index')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Over Images')
plt.legend()
plt.show()
"""

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


real pics training
Text generated from image facebook.com:
facebook.com
The edit distance between 'facebook.com' and 'facebook.com' is: 0
Text generated from image airporthandling.eu:
airporthandling.eu
The edit distance between 'airporthandling.eu' and 'airporthandling.eu' is: 0
Text generated from image zcsazzurrowebportal.com:
zcsazzurrowebportal.
The edit distance between 'zcsazzurrowebportal.com' and 'zcsazzurrowebportal.' is: 3
Text generated from image bing.com:
bing.com.
The edit distance between 'bing.com' and 'bing.com.' is: 1
Text generated from image mediaset.it:
mediaset.it
The edit distance between 'mediaset.it' and 'mediaset.it' is: 0
Text generated from image ilfattoquotidiano.it:
ilfattoquotidiano.it
The edit distance between 'ilfattoquotidiano.it' and 'ilfattoquotidiano.it' is: 0
Text generated from image twitter.com:
twitter.com
The edit distance between 'twitter.com' and 'twitter.com' is: 0
Text generated from image transfermarkt.it:
transfermarkt.it
The edit distan

'\nprint(\'fake pics training\')\nfor item in os.listdir(folder_path_fake_pics):\n    if item.endswith(\'.jpg\'):\n        image_path = os.path.join(folder_path_fake_pics, item)\n        image = Image.open(image_path)\n\n        pixel_values = processor(image, return_tensors="pt").pixel_values\n\n        generated_ids = model.generate(pixel_values)\n        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]\n\n        generated_text = generated_text.replace(" ", "")\n        item = item[:-4]\n\n        edit_distance = calculate_word_accuracy(item, generated_text)\n\n        # Calcola la precisione\n        word_accuracy = 100 * (1 - (edit_distance / max(len(item), len(generated_text))))\n        \n        # Calcola la precisione media\n        word_count += 1\n        average_accuracy = ((word_count - 1) * average_accuracy + word_accuracy) / word_count\n\n        word_accuracies.append(word_accuracy)\n        average_accuracies.append(average_accuracy)\